In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm 
import torch

/home/dazhou/anaconda3/envs/calibration/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/Checkin.tsv', sep='\t')

In [5]:
df

,UserId,CheckinTime,VenueId,VenueType,X,Y
0,1058,2019-07-31 00:00:00,13774,Apartment,2.401848e+06,470851.019004
1,1148,2019-07-31 00:00:00,17662,Pub,2.400037e+06,470943.228524
2,1157,2019-07-31 00:00:00,3128,Apartment,2.400599e+06,474295.434912
3,1329,2019-07-31 00:00:00,3572,Apartment,2.401035e+06,474095.666130
4,1446,2019-07-31 00:00:00,5825,Apartment,2.402679e+06,474025.455325
...,...,...,...,...,...,...
1683920,9341,2019-08-30 00:05:00,17755,Restaurant,2.399608e+06,473685.151488
1683921,9531,2019-08-30 00:05:00,17728,Restaurant,2.402893e+06,473297.278125
1683922,9535,2019-08-30 00:05:00,10176,Apartment,2.402105e+06,471693.433545
1683923,9735,2019-08-30 00:05:00,17770,Restaurant,2.402764e+06,472885.268209


In [4]:
df['CheckinTime'] = pd.to_datetime(df['CheckinTime'])

In [6]:
df_train = df.loc[df.CheckinTime < pd.to_datetime("2019-08-15T00:00:00.000")]
df_test = df.loc[df.CheckinTime >= pd.to_datetime("2019-08-15T00:00:00.000")]

In [7]:
type_train = df_train.pivot(columns='CheckinTime', index='UserId', values='VenueType')
type_test = df_test.pivot(columns='CheckinTime', index='UserId', values='VenueType')

In [21]:
print((type_train).shape,(type_test).shape)

(9506, 3479) (9506, 3476)


In [11]:
X_train = df_train.pivot(columns='CheckinTime', index='UserId', values='X')
X_test = df_test.pivot(columns='CheckinTime', index='UserId', values='X')
Y_train = df_train.pivot(columns='CheckinTime', index='UserId', values='Y')
Y_test = df_test.pivot(columns='CheckinTime', index='UserId', values='Y')

In [14]:
X_train

CheckinTime,2019-07-31 00:00:00,2019-07-31 00:05:00,2019-07-31 00:10:00,2019-07-31 00:15:00,2019-07-31 00:20:00,2019-07-31 00:25:00,2019-07-31 00:30:00,2019-07-31 00:35:00,2019-07-31 00:40:00,2019-07-31 00:45:00,...,2019-08-14 23:10:00,2019-08-14 23:15:00,2019-08-14 23:20:00,2019-08-14 23:25:00,2019-08-14 23:30:00,2019-08-14 23:35:00,2019-08-14 23:40:00,2019-08-14 23:45:00,2019-08-14 23:50:00,2019-08-14 23:55:00
UserId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.402464e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.402464e+06
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
traj_train = np.concatenate([X_train.values, Y_train.values],axis=-1).reshape(-1, 2, X_train.shape[1])
traj_test = np.concatenate([X_test.values, Y_test.values],axis=-1).reshape(-1, 2, X_test.shape[1])

In [18]:
traj_train.shape

(9506, 2, 3479)

In [15]:
traj_train.shape, traj_test.shape, type_train.shape, type_test.shape

((9506, 2, 3479), (9506, 2, 3476), (9506, 3479), (9506, 3476))

In [16]:
traj_train_tensor = torch.tensor(traj_train, dtype=torch.float).permute((0,2,1))
traj_test_tensor = torch.tensor(traj_test, dtype=torch.float).permute((0,2,1))

In [17]:
print(traj_train_tensor.shape,traj_test_tensor.shape)

torch.Size([9506, 3479, 2]) torch.Size([9506, 3476, 2])
